In [1]:
from rdkit import Chem
import torch
torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

In [2]:
# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print(device)
in_path = '/scratch/20cy91r19/bitbucket/afirxn/afirxn/AIMNet2/data/examples.sdf'

aimnet2 = torch.jit.load('/scratch/20cy91r19/bitbucket/afirxn/afirxn/AIMNet2/models/aimnet2_wb97m-d3_ens.jpt', map_location=device)
aimnet2_0 = torch.jit.load('/scratch/20cy91r19/bitbucket/afirxn/afirxn/AIMNet2/models/aimnet2_wb97m-d3_0.jpt', map_location=device)

cpu


In [3]:
def sdf2aimnet_input(sdf: str, device=torch.device('cpu')) -> dict:
    """Converts sdf to aimnet input, assuming the sdf has only 1 conformer."""
    mol = next(Chem.SDMolSupplier(sdf, removeHs=False))
    conf = mol.GetConformer()
    coord = torch.tensor(conf.GetPositions(), device=device).unsqueeze(0)
    numbers = torch.tensor([atom.GetAtomicNum() for atom in mol.GetAtoms()], device=device).unsqueeze(0)
    charge = torch.tensor([Chem.GetFormalCharge(mol)], device=device, dtype=torch.float)
    return dict(coord=coord, numbers=numbers, charge=charge)


dct = sdf2aimnet_input(in_path, device=device)
dct['coord'].requires_grad_(True)
aimnet2_out = aimnet2(dct)
print('aimnet2 energy: ', aimnet2_out['energy'])  # there is no gradient for energy

aimnet2 energy:  tensor([-10518.2732], dtype=torch.float64)


In [4]:
for key, val in aimnet2_out.items():
    print(key, val.shape)

coord torch.Size([1, 20, 3])
numbers torch.Size([1, 20])
charge torch.Size([1])
charges torch.Size([1, 20])
charges_std torch.Size([1, 20])
energy torch.Size([1])
energy_std torch.Size([1])
forces torch.Size([1, 20, 3])
forces_std torch.Size([1, 20, 3])


In [5]:
print('aimnet2 energy: ', aimnet2_out['energy'])

aimnet2 energy:  tensor([-10518.2732], dtype=torch.float64)


In [6]:
aimnet2_0out = aimnet2_0(dct)
print('aimnet2_0 energy: ', aimnet2_0out['energy'])  # there is gradient function for energy

aimnet2_0 energy:  tensor([-10518.2655], dtype=torch.float64, grad_fn=<AddBackward0>)


In [7]:
forces = -torch.autograd.grad(aimnet2_0out['energy'], dct['coord'], create_graph=True)[0]
print(forces)

tensor([[[ 0.0198,  0.1783, -0.0657],
         [ 0.0356, -0.2943,  0.0706],
         [-0.0442,  0.0613,  0.1326],
         [ 0.1627,  0.1406,  0.0440],
         [-0.0146, -0.1142, -0.1864],
         [-0.1602,  0.0205, -0.0834],
         [-0.3099, -0.3564, -0.0733],
         [ 0.0133, -0.1086,  0.0467],
         [-0.1510,  0.0309, -0.0501],
         [-0.1301,  0.0278, -0.1938],
         [ 0.1394,  0.1691, -0.0113],
         [ 0.1099, -0.1648, -0.1143],
         [-0.0429, -0.0140, -0.2180],
         [-0.1936,  0.0702,  0.0784],
         [ 0.0217, -0.1661,  0.0584],
         [ 0.1824,  0.0953,  0.2214],
         [-0.1041,  0.1576,  0.1365],
         [ 0.0342,  0.0051,  0.1738],
         [ 0.1627, -0.0603, -0.0591],
         [ 0.2689,  0.3220,  0.0929]]], dtype=torch.float64,
       grad_fn=<NegBackward0>)
